In [ ]:
import os
import shutil
import tarfile
import tensorflow as tf
import pandas as pd
from bs4 import BeautifulSoup
import re
import sys
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.callbacks import EarlyStopping
from scipy.special import softmax
import numpy as np
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, BatchEncoding, BertConfig
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import tf_keras
!pip install datasets
import datasets
from datasets import load_dataset
from transformers import BatchEncoding
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (1)
sst2_dataset = load_dataset("glue", "sst2")
train_dataset = sst2_dataset["train"]
validation_dataset = sst2_dataset["validation"]
test_dataset = sst2_dataset["test"]
train_df=pd.DataFrame(train_dataset.to_pandas())
val_df=pd.DataFrame(validation_dataset.to_pandas())
test_df=pd.DataFrame(test_dataset.to_pandas())
df=pd.concat([train_df, val_df], ignore_index=True)
df.drop(columns=['idx'], inplace=True)
print(df)

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (2)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['label'], random_state=5)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['label'], random_state=5)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df, unlab_df= train_test_split(train_df, test_size=0.4, stratify=train_df['label'], random_state=5)
unlab_df.loc[:, 'label']=2
print(train_df)
print(unlab_df)
print(val_df)
print(test_df)
#For the first split, put the frac value below to be equal to 0.0031 for about 100 labeled samples
#For the second split, put the frac value below to be equal to 0.01 for about 300 labeled samples
#For the third split, put the frac value below to be equal to 0.0153 for about 500 labeled samples
#For the fourth split, put the frac value below to be equal to 0.031 for about 1000 labeled samples
#For the fifth split put the frac value below to be equal to 0.0765 for about 2500 labeled samples
#For the sixth split, put the frac value below to be equal to 0.153 for about 5000 labeled samples
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.0153))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE IMDB Movie Review DATASET
cwd=os.getcwd()
data=tf.keras.utils.get_file(fname ="aclImdb.tar.gz", origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", cache_dir= cwd, extract = True)
data_path=os.path.join(os.path.dirname(data), 'aclImdb')
os.listdir(data_path)
train_dir=os.path.join(data_path, 'train')
for file in os.listdir(train_dir):
    file_path=os.path.join(train_dir, file)
    if os.path.isfile(file_path):
        with open(file_path) as f:
            first_line=f.readline().strip()
            print(file + ":" + first_line)
def load_dataset(dir, typeOfData):
    dictnry = {"review": [], "sentiment": []}
    if typeOfData == 'labelled':
        pos_path=os.path.join(dir, 'pos')
        neg_path=os.path.join(dir, 'neg')
        for file in os.listdir(pos_path):
            file_path=os.path.join(pos_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(1)
        for file in os.listdir(neg_path):
            file_path=os.path.join(neg_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(0)
    elif typeOfData == 'unsup':
        unsup_path=os.path.join (dir, 'unsup')
        for file in os.listdir(unsup_path):
            file_path=os.path.join(unsup_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(2)
    return dictnry
Train_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'labelled'))
Unlab_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'unsup'))
test_dir=os.path.join(data_path, 'test')
Test_df=pd.DataFrame.from_dict(load_dataset(test_dir, 'labelled'))
df=pd.concat([Train_df, Test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['sentiment'], random_state=42)
train_df, unlab_df= train_test_split(train_df, test_size=0.375, stratify=train_df['sentiment'], random_state=42)
unlab_df.loc[:, 'sentiment']=2
#For the first split, put the frac value from the train_df below to be equal to 0.004 for about 100 labeled
#For the second split, put the frac value from the train_df below to be equal to 0.012 for about 300 labeled
#For the third split, put the frac value from the train_df below to be equal to 0.04 for about 1000 labeled
#For the fourth split, put the frac value from the train_df below to be equal to 0.2 for about 5000 labeled
train_df=train_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.2))
val_df=val_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
test_df=test_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (1)
from google.colab import files
uploaded = files.upload()

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (2)
train_df=pd.read_csv('/content/Corona_NLP_train.csv', encoding='ISO-8859-1')
test_df=pd.read_csv('/content/Corona_NLP_test.csv', encoding='ISO-8859-1')
train_df=train_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
test_df=test_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
train_df.drop_duplicates(subset='OriginalTweet',inplace=True)
test_df.drop_duplicates(subset='OriginalTweet',inplace=True)
df=pd.concat([train_df, test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['Sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['Sentiment'], random_state=42)
mapping = {'Extremely Negative': 0, 'Negative': 1, 'Neutral': 2, 'Positive':3, 'Extremely Positive':4}
train_df['Sentiment']=train_df['Sentiment'].replace(mapping)
val_df['Sentiment']=val_df['Sentiment'].replace(mapping)
test_df['Sentiment']=test_df['Sentiment'].replace(mapping)
train_df['Sentiment']=train_df['Sentiment'].astype('category')
val_df['Sentiment']=val_df['Sentiment'].astype('category')
test_df['Sentiment']=test_df['Sentiment'].astype('category')
train_df, unlab_df= train_test_split(train_df, test_size=0.42, stratify=train_df['label'], random_state=42)
unlab_df['Sentiment']=unlab_df['Sentiment'].cat.add_categories(5)
unlab_df['Sentiment']=5
#For the first split, put the frac value below to be equal to 0.0051
#For the second split, put the frac value below to be equal to 0.0255
#For the third split, put the frac value below to be equal to 0.051
#For the fourth split, put the frac value below to be equal to 0.2545
train_df=train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.0051))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT SST-5 DATASET
dataset=load_dataset("SetFit/sst5")
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']
train_df=pd.DataFrame(train_dataset)
train_df=train_df.drop(columns=['label_text'])
val_df=pd.DataFrame(val_dataset)
val_df=val_df.drop(columns=['label_text'])
test_df=pd.DataFrame(test_dataset)
test_df=test_df.drop(columns=['label_text'])
train_df, unlab_df= train_test_split(train_df, test_size=0.53, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=5
#For the first split, put the frac value below to be equal to 0.027
#For the second split, put the frac value below to be equal to 0.125
#For the third split, put the frac value below to be equal to 0.25
#For the fourth split, put the frac value below to be equal to 0.5
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.125))
train_df.drop_duplicates(subset='text',inplace=True)
unlab_df.drop_duplicates(subset='text', inplace=True)
val_df.drop_duplicates(subset='text',inplace=True)
test_df.drop_duplicates(subset='text',inplace=True)
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
def cleanText(text):
    soup=BeautifulSoup(text, "html.parser")
    text=re.sub(r"\[[^]]*\]" , '', soup.get_text())
    text=re.sub(r"[^A-Za-z0-9\s,']", '', text)
    return text

In [ ]:
train_df['cleaned']=train_df[train_df.columns[0]].apply(cleanText)
unlab_df['cleaned']=unlab_df[train_df.columns[0]].apply(cleanText)
val_df['cleaned']=val_df[train_df.columns[0]].apply(cleanText)
test_df['cleaned']=test_df[train_df.columns[0]].apply(cleanText)

In [ ]:
max_len=64
tz=BertTokenizer.from_pretrained('bert-base-uncased')
train_encoded=tz.batch_encode_plus(train_df['cleaned'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
unlab_encoded=tz.batch_encode_plus(unlab_df['cleaned'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
val_encoded=tz.batch_encode_plus(val_df['cleaned'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
test_encoded=tz.batch_encode_plus(test_df['cleaned'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

In [ ]:
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
TR1=train_encoded['input_ids']
TR2=train_encoded['attention_mask']
U1=unlab_encoded['input_ids']
U2=unlab_encoded['attention_mask']
V1=val_encoded['input_ids']
V2=val_encoded['attention_mask']
TS1=test_encoded['input_ids']
TS2=test_encoded['attention_mask']

In [ ]:
trainY=tf.convert_to_tensor(train_df[train_df.columns[1]])
unique, _=tf.unique(trainY)
trainY=tf.one_hot(trainY, depth=unique.shape[0])
unlabY=tf.convert_to_tensor(unlab_df[unlab_df.columns[1]])
valY=tf.convert_to_tensor(val_df[val_df.columns[1]])
valY=tf.one_hot(valY, depth=unique.shape[0])
testY=tf.convert_to_tensor(test_df[test_df.columns[1]])
testY=tf.one_hot(testY, depth=unique.shape[0])

In [ ]:
class CustomModel(tf_keras.Model):
    def __init__(self, num_classes=trainY.shape[1]):
        super().__init__()
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased', num_attention_heads=12)
        #j, the current number of iteration
        iter=1 #the number of iterations for which the first dropout value will be applied. You change it according to the experiments in the pdf
        if j < iter:
            self.dropout=tf_keras.layers.Dropout(0.2)
        elif j >= iter:
            self.dropout=tf_keras.layers.Dropout(0.2)
        self.layer1=tf_keras.layers.Dense(units=num_classes)

    def call(self, inputs, training=True):
        inputIds, attentionMask=inputs
        embeddings=self.bert_model(inputIds, attentionMask)['last_hidden_state']
        x=embeddings[:, 0]
        if clsCount == 1:
            x=tf.gather(x, random_indexes[0:random_indexes.shape[0]//2], axis=1)
            print(x)
        elif clsCount == 2:
            x=tf.gather(x, random_indexes[random_indexes.shape[0]//2:random_indexes.shape[0]], axis=1)
            print(x)
        pooled_output=x
        if training:
            pooled_output = self.dropout(x, training=training)
        logits = self.layer1(pooled_output)
        return logits

In [ ]:
global random_indexes
global clsCount #the variable which controls which view the current model, we have defined, takes
random_indexes=tf.random.shuffle(tf.range(1, 769))
global j
j=0 # the current iteration
ogSize=U1.shape[0]
print(ogSize)
while U1.shape[0] != 0:
    model1=CustomModel()
    model2=CustomModel()
    num_epochs=100
    if trainY.shape[1] == 2:
        loss1=tf_keras.losses.BinaryCrossentropy(from_logits=True)
        loss2=tf_keras.losses.BinaryCrossentropy(from_logits=True)
        metric1=tf_keras.metrics.BinaryAccuracy('accuracy')
        metric2=tf_keras.metrics.BinaryAccuracy('accuracy')
    elif trainY.shape[1] == 5:
        loss1=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
        loss2=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
        metric1=tf_keras.metrics.CategoricalAccuracy('accuracy')
        metric2=tf_keras.metrics.CategoricalAccuracy('accuracy')
    if j == 0:
        initial_lr=1e-5
        opt1=tf_keras.optimizers.Adam(learning_rate=initial_lr)
        opt2=tf_keras.optimizers.Adam(learning_rate=initial_lr)
        callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    if j > 0:
        lr=9e-6
        opt1=tf_keras.optimizers.Adam(learning_rate=lr)
        opt2=tf_keras.optimizers.Adam(learning_rate=lr)
        callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    model1.compile(optimizer=opt1, loss=loss1, metrics=[metric1])
    model2.compile(optimizer=opt2, loss=loss2, metrics=[metric2])
    clsCount=1
    model1.fit([TR1, TR2], trainY, batch_size=32, epochs=50, shuffle=True, callbacks=[callback], verbose=1, validation_data=([V1, V2], valY))
    logits1=model1.predict([U1, U2])
    logits1=tf.convert_to_tensor(logits1)
    if trainY.shape[1] == 2:
        prob1=tf.reduce_max(tf.nn.sigmoid(logits1), axis=1)
        pred1=tf.one_hot(tf.argmax(tf.nn.sigmoid(logits1), axis=1), depth=trainY.shape[1])
    elif trainY.shape[1] == 5:
        prob1=tf.reduce_max(tf.nn.softmax(logits1, axis=1), axis=1)
        pred1=tf.one_hot(tf.argmax(tf.nn.softmax(logits1, axis=1), axis=1), depth=trainY.shape[1])
    clsCount=2
    model2.fit([TR1, TR2], trainY, batch_size=32, epochs=50, shuffle=True, callbacks=[callback], verbose=1, validation_data=([V1, V2], valY))
    logits2=model2.predict([U1, U2])
    logits2=tf.convert_to_tensor(logits2)
    if trainY.shape[1] == 2:
        prob2=tf.reduce_max(tf.nn.sigmoid(logits2), axis=1)
        pred2=tf.one_hot(tf.argmax(tf.nn.sigmoid(logits2), axis=1), depth=trainY.shape[1])
    elif trainY.shape[1] == 5:
        prob2=tf.reduce_max(tf.nn.softmax(logits2, axis=1), axis=1)
        pred2=tf.one_hot(tf.argmax(tf.nn.softmax(logits2, axis=1), axis=1), depth=trainY.shape[1])
    indexes=[]
    count=0
    p=0.85 #the threshold which the max predicted probability of a sample must overcome so that it be added to the training data
    for i in range(prob1.shape[0]):
        if trainY.shape[1] == 2:
            logits1_after_activ=tf.nn.sigmoid(logits1[i])
            logits2_after_activ=tf.nn.sigmoid(logits2[i])
        elif trainY.shape[1] == 5:
            logits1_after_activ=tf.nn.softmax(logits1[i])
            logits2_after_activ=tf.nn.softmax(logits2[i])
        if (prob1[i] >= p or prob2[i] >= p):
            if (prob1[i] >= p and prob2[i] >= p):
                if tf.argmax(logits1_after_activ).numpy() == tf.argmax(logits2_after_activ).numpy():
                    count+=1
                    indexes.append(i)
                    TR1=tf.concat([TR1, tf.reshape(U1[i], (1, U1[i].shape[0]))], axis=0)
                    TR2=tf.concat([TR2, tf.reshape(U2[i], (1, U2[i].shape[0]))], axis=0)
                    trainY=tf.concat([trainY, tf.reshape(pred1[i], (1, trainY.shape[1]))], axis=0)
            else:
                count+=1
                indexes.append(i)
                TR1=tf.concat([TR1, tf.reshape(U1[i], (1, U1[i].shape[0]))], axis=0)
                TR2=tf.concat([TR2, tf.reshape(U2[i], (1, U2[i].shape[0]))], axis=0)
                if prob1[i] > prob2[i]:
                    trainY=tf.concat([trainY, tf.reshape(pred1[i], (1, trainY.shape[1]))], axis=0)
                elif prob2[i] >= prob1[i]:
                    trainY=tf.concat([trainY, tf.reshape(pred2[i], (1, trainY.shape[1]))], axis=0)
    print(TR1.shape)
    print(trainY.shape)
    if len(indexes) != 0:
        maskX=tf.ones(U1.shape[0], tf.bool)
        maskY=tf.ones(unlabY.shape[0], tf.bool)
        indices_to_update=tf.expand_dims(indexes, axis=1)
        updatesX=tf.zeros_like(indexes, dtype=tf.bool)
        updatesY=tf.zeros_like(indexes, dtype=tf.bool)
        maskX=tf.tensor_scatter_nd_update(maskX, indices_to_update, updatesX)
        maskY=tf.tensor_scatter_nd_update(maskY, indices_to_update, updatesY)
        U1=tf.boolean_mask(U1, maskX)
        U2=tf.boolean_mask(U2, maskX)
        unlabY=tf.boolean_mask(unlabY, maskY)
        print(U1.shape)
    print(count, "samples were added")
    j+=1
    if j == 3:
        break

In [ ]:
#NEW
clsCount=1
if trainY.shape[1] == 2:
    test_probs1=tf.nn.sigmoid(tf.convert_to_tensor(model1.predict([TS1, TS2])))
elif trainY.shape[1] == 5:
    test_probs1=tf.nn.softmax(tf.convert_to_tensor(model1.predict([TS1, TS2])), axis=1)
test_preds1=tf.argmax(test_probs1, axis=1)
clsCount=2
if trainY.shape[1] == 2:
    test_probs2=tf.nn.sigmoid(tf.convert_to_tensor(model2.predict([TS1, TS2])))
elif trainY.shape[1] == 5:
    test_probs2=tf.nn.softmax(tf.convert_to_tensor(model2.predict([TS1, TS2])), axis=1)
test_preds2=tf.argmax(test_probs2, axis=1)
test_preds1=test_preds1.numpy()
test_preds2=test_preds2.numpy()
test_pred1_labels=tf.one_hot(tf.convert_to_tensor(test_preds1), depth=trainY.shape[1])
test_pred2_labels=tf.one_hot(tf.convert_to_tensor(test_preds2), depth=trainY.shape[1])
if trainY.shape[1] == 2:
    acc1=tf_keras.metrics.BinaryAccuracy()
    acc2=tf_keras.metrics.BinaryAccuracy()
elif trainY.shape[1] == 5:
    acc1=tf_keras.metrics.CategoricalAccuracy()
    acc2=tf_keras.metrics.CategoricalAccuracy()
acc1.update_state(testY, test_pred1_labels)
acc2.update_state(testY, test_pred2_labels)
avg_acc=tf.reduce_mean([acc1.result().numpy(), acc2.result().numpy()])
print(avg_acc.numpy())

In [ ]:
#THE SUPERVISED MODEL FOR THE SAME DATA.IF YOU HAVE RAN THE SEMI SUPERVISED MODEL FIRST, THEN YOU HAVE TO RUN FIRST THE CELLS WHICH START WITH THE DEFINITION OF TRAIN_DF['cleaned'] UNTIL THE SEMI SUPERVISED MODEL.THEN RUN THIS CELL
config=BertConfig.from_pretrained('bert-base-uncased')
config.num_labels=trainY.shape[1]
config.hidden_dropout_prob=0.2
modelSup=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
lr=1e-5
num_epochs=100
if trainY.shape[1] == 2:
  lossSup=tf_keras.losses.BinaryCrossentropy(from_logits=True)
  metricSup=tf_keras.metrics.BinaryAccuracy('accuracy')
elif trainY.shape[1] == 5:
  lossSup=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
  metricSup=tf_keras.metrics.CategoricalAccuracy('accuracy')
optSup=tf_keras.optimizers.Adam(learning_rate=lr)
callbackSup=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
metricSup=tf_keras.metrics.CategoricalAccuracy('accuracy')
modelSup.compile(optimizer=optSup, loss=lossSup, metrics=[metricSup])
modelSup.fit([TR1, TR2], trainY, batch_size=32, epochs=num_epochs, shuffle=True, callbacks=[callbackSup], verbose=1, validation_data=([V1, V2], valY))

In [ ]:
test_loss, test_accuracy = modelSup.evaluate([TS1, TS2], testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')